In [ ]:
import math
import itertools
import pandas as pd

In [ ]:
# elements without isotope pattern in mass spectra
elements = ["C","H", "O", "N", "P", "F", "I"]

# elements with isotope patterns in mass spectra
isotope_pattern_elements = ["Cl", "Br", "S", "Si", "B"]

In [ ]:
# only for preparing the csv file

'''
import periodictable as pse

masses = {
    "C" : pse.carbon.mass, 
    "H" : pse.hydrogen.mass,
    "N" : pse.nitrogen.mass,
    "O" : pse.oxygen.mass,
    "F" : pse.fluorine.mass,
    "Cl" : pse.chlorine.mass,
    "Br" : pse.bromine.mass,
    "I" : pse.iodine.mass,
    "Si" : pse.silicon.mass,
    "P" : pse.phosphorus.mass,
    "S" : pse.sulfur.mass,
    "B" : pse.boron.mass
}

with open("element_masses.csv", "a") as f:
    print("element, mass", file=f)
for element in element_symbols:
    with open("element_masses.csv", "a") as f:
        print(f"{element}, {masses[elem]:.5f}", file=f)
'''

In [ ]:
# exact mass of the mass peak
mass = 98.0

# tolerance for deviation between predicted formula mass and given mass
tol = 0.1

# read element masses from csv file
elements_masses = pd.read_csv("element_masses.csv", sep=",", index_col="element")

In [ ]:
# some elements always have to be considered, therefore just copy them over
possible_elements = elements.copy()
# for smoothly handling false input
isotope_pattern_choice = None

while not (isotope_pattern_choice == "Yes" or isotope_pattern_choice == "No"):
        
    # user decides has to tell if there is an isotope pattern
    isotope_pattern_choice = input("Isotope pattern (Yes/No)?: ")

    # if there is an isotope patter in the spectrum collect the corresponding elements
    if isotope_pattern_choice == "Yes":

        for element in isotope_pattern_elements:
                
            # for smoothly handling false input
            element_choice = None

            while not (element_choice == "Yes" or element_choice == "No"):
                    
                # user has to tell if isotope pattern corresponds to current element
                element_choice = input(f"{element} (Yes/No)?: ")

                if element_choice == "Yes":
                        
                    possible_elements.append(element)

In [ ]:
num_elements = {}

for element in possible_elements:

    # get (mathematically) highest possible number of element in the molecule given the total mass
    num_element = math.ceil(mass / elements_masses.loc[element].iloc[0])
        
    # store all numbers {0, ..., highest possible number}
    num_elements[element] = [i for i in range(num_element)]

In [ ]:
# collect all predicted formulas in list
formulas = []
# convert possible numbers of elements to list to create combinations
n = list(num_elements.values())

# loop over all possible combinations of numbers of the different elements
for combination in itertools.product(*n):

    formula = ""

    # to compare against mass peak within tolerance
    formula_mass = 0

    # "double bound equivalents" to check if formula is chemically reasonable
    dbe = 0

    # index i corresponds to the index of the element in possible_elements, n to the highest possible number
    for i, n in enumerate(combination):
        
        # element not in molecule, don't to formula and continue with next element
        if n == 0:
            continue

        element = possible_elements[i]

        # leave out index 1 for elements in formula
        if n > 1:
            formula += f"{element}{n}"
        else:
            formula += element

        # add respective element mass to total formula mass
        formula_mass += n * elements_masses.loc[element].iloc[0]

        # add respective contributation to double bound equivalent
        if element in ["C", "Si"]:
            dbe += 2 * n
        elif element in ["B", "N", "P"]:
            dbe += n
        elif element in ["H", "F", "Cl", "Br", "I"]:
            dbe -= n

    # single O and S atom not reasonable but have to be checked individually
    if formula == "O" or formula == "S":
        continue

    # if deviation to mass peak higher than chosen tolerance discard the formula
    if abs(formula_mass - mass) > tol:
        continue

    dbe = (dbe + 2) / 2
    # reasonable chemical formulas have positive integer double bound equivalents
    if dbe < 0:
        continue
    if dbe % 1 != 0:
        continue

    # if all checks passed store formula as potential candidate
    formulas.append(formula)

In [ ]:
print("Possible molecular formulas are:", end="\n\n")
for formula in formulas:
    print(formula)